System Integration II 
Author : Tuguldur Tserenbajir & Jason Theodorus
Date : 11th January 2024 

### Initializations 

In [11]:
#IMPORT LIBRARIES   

import pandas as pd
from sklearn.model_selection import train_test_split
import openai


In [12]:
api_key = ""
#Read Dataset from Excel Workbook
df = pd.read_excel('Amazon_Train.xlsx')

# Display the first few rows of the DataFrame to confirm it's loaded correctly
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Label,Aspect(Pos),Aspect(Neg),ProductTitle,length
0,42.0,B001EO5QW8,A1WK4ALVZDYPUE,"Dick Baldwin ""christobe""",0.0,0.0,5.0,1.302134e+09,"McCann's offers a variety of oatmeal options, ...",McCann's makes oatmeal for every oatmeal conno...,positive,"oatmeal, variety, McCann","bulk only, thick, liquidy","McCANN'S Instant Irish Oatmeal, Variety Pack o...",226.0
1,53.0,B000G6RPMY,A9L6L5H9BPEBO,Edwin C. Pauzer,1.0,1.0,4.0,1.348877e+09,The writer found various items at a specific s...,This wasn't in stock the last time I looked. I...,positive,"gift, Ass Kickin Peanuts","extreme spiciness,",Ass Kickin' Peanuts - With Habanero,220.0
2,74.0,B0059WXJKM,A25VFHVGI4CFTP,Diana Robinson,0.0,1.0,1.0,1.335744e+09,"Maltitol, a sugar substitute, can cause digest...",Buyer Beware Please! This sweetener is not for...,negative,maltitol,"nausea, diarrhea, headache, caution, bloating,...",Maltitol,306.0
3,113.0,B0037LW78C,A2F65U0UDY7TAW,Aldo,2.0,2.0,5.0,1.337990e+09,I switched from drinking coffee to Royal King ...,I have been drinking Royal King 100% Natural O...,positive,"everyday tea, green tea, health benefit","packaging, tea bag ripping",Royal King 100% Natural Organic Green Tea,334.0
4,119.0,B003SE19UK,AD1WWGMEQ8FWW,Bob Rex,5.0,5.0,5.0,1.290989e+09,I tried a new cat food for my two cats. The yo...,I had heard a little about this product from t...,positive,"healthy, alternative, diet change",expensive,"Holistic Select Natural Dry Cat Food, Duck, 5....",298.0


##### Convert Dataframe to JSONL for finetuning

In [13]:
#Filter the data needed for finetuning
finetune_df = df[['Summary', 'Label', 'Aspect(Pos)', 'Aspect(Neg)']]
finetune_df.columns = ['summary', 'sentiment', 'aspect_pos', 'aspect_neg']

#get only the first 200 lines and save to new df


##### Split for Training and Testing

## Zero Shot Approach (Base Prompting)

I tried prompting on ChatGPT 3.5 Turbo model and the result is as  so

Prompt 1 : 

Provide me Sentiment, Positive Aspects, Negative Aspects . Sentiment should only be positive/negative/neutral. Aspects should be 3 words maximum. If there are no Positive/Negative Aspects provide the string "x". The format is as so : 

Sentiment: {sentiment}

Aspect(Pos): {aspect_pos}

Aspect(Neg): {aspect_neg}

Prompt 2 :

McCann's offers a variety of oatmeal options, including instant flavors. It cooks up thicker than other brands but may be a bit liquidy with Apples & Cinnamon flavor. You need to buy in bulk, which is great for families.

Result Prompt 2 :

<strong>
Sentiment: Positive

Aspect(Pos): Variety, Families

Aspect(Neg): Liquidy
</strong>

Prompt 3 :

Our 10-month-old twins love this baby food with organic apples, carrots, brown rice, and lentils. It provides a complete protein despite lentils not being a prime ingredient. They enjoy the sweet potato and fruit varieties too. The packaging is excellent, with reliable delivery. It's not available in local stores, so we appreciate the subscription service. Each serving has 70 calories and provides essential nutrients. Note that the daily intake percentage is based on an adult.

Result Prompt 3 :

<strong>
Sentiment: Positive

Aspect(Pos): Organic, Packaging, Variety

Aspect(Neg): x
</strong>




#### GPT 3.5 Turbo (Fine Tuning)

##### Data Wrangling for finetuning GPT 3.5 Turbo

In [14]:
# Function to create a formatted completion string
def create_completion(row):
    sentiment = f"Sentiment: {row['sentiment'].capitalize()}"  # Capitalize the sentiment
    aspect_pos = f"Aspect(Pos): {row['aspect_pos']}"
    aspect_neg = f"Aspect(Neg): {row['aspect_neg'] if row['aspect_neg'] != 'x' else 'None'}"  # Handle 'x' as 'None'
    return f"{sentiment}\n{aspect_pos}\n{aspect_neg}"

# Apply the function to create the completion column
finetune_df['completion'] = finetune_df.apply(create_completion, axis=1)

# Create the prompt column
finetune_df['prompt'] = "Provide me Sentiment, Positive Aspects, Negative Aspects . Sentiment should only be positive/negative/neutral. List more than 1 aspect, but each aspect should be 3 words max. If there are no Positive/Negative Aspects provide the string x. Do it for the following: " + finetune_df['summary']

finetune_df.head()

C:\Users\Jet\AppData\Local\Temp\ipykernel_2456\3800243770.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finetune_df['completion'] = finetune_df.apply(create_completion, axis=1)
C:\Users\Jet\AppData\Local\Temp\ipykernel_2456\3800243770.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finetune_df['prompt'] = "Provide me Sentiment, Positive Aspects, Negative Aspects . Sentiment should only be positive/negative/neutral. List more than 1 aspect, but each aspect should be 3 words max. If there are no Po

,summary,sentiment,aspect_pos,aspect_neg,completion,prompt
0,"McCann's offers a variety of oatmeal options, ...",positive,"oatmeal, variety, McCann","bulk only, thick, liquidy","Sentiment: Positive\nAspect(Pos): oatmeal, var...","Provide me Sentiment, Positive Aspects, Negati..."
1,The writer found various items at a specific s...,positive,"gift, Ass Kickin Peanuts","extreme spiciness,","Sentiment: Positive\nAspect(Pos): gift, Ass Ki...","Provide me Sentiment, Positive Aspects, Negati..."
2,"Maltitol, a sugar substitute, can cause digest...",negative,maltitol,"nausea, diarrhea, headache, caution, bloating,...",Sentiment: Negative\nAspect(Pos): maltitol\nAs...,"Provide me Sentiment, Positive Aspects, Negati..."
3,I switched from drinking coffee to Royal King ...,positive,"everyday tea, green tea, health benefit","packaging, tea bag ripping",Sentiment: Positive\nAspect(Pos): everyday tea...,"Provide me Sentiment, Positive Aspects, Negati..."
4,I tried a new cat food for my two cats. The yo...,positive,"healthy, alternative, diet change",expensive,"Sentiment: Positive\nAspect(Pos): healthy, alt...","Provide me Sentiment, Positive Aspects, Negati..."


##### Split for Training and Testing

In [15]:
train_df, test_df = train_test_split(finetune_df, test_size=0.2, random_state=42)



# Convert training data to JSON Lines with new prompts and completions
train_df[['prompt', 'completion']].to_json('train_data_with_formatted_completions.jsonl', orient='records', lines=True)

# Convert testing data to JSON Lines with new prompts and completions
test_df[['prompt', 'completion']].to_json('test_data_with_formatted_completions.jsonl', orient='records', lines=True)

print(train_df.shape)
print(test_df.shape)


(655, 6)
(164, 6)


#### Conduct Finetuning

In [16]:
# openai.api_key = 'your-api-key'

# # Assuming you've correctly formatted and uploaded your chat-formatted JSONL file
# training_file = 'path/to/chat_formatted_data.jsonl'

# # Create a fine-tuning job using the chat model
# response = openai.FineTune.create(
#     training_file=training_file,
#     model="gpt-3.5-turbo-0125",
#     n_epochs=4,
#     batch_size=4,
#     learning_rate_multiplier=0.1
# )

# print("Fine-tune job ID:", response['id'])


#### Inference Model

In [17]:
#Single Inference

# Set up your API key
openai.api_key = ''

# Model ID of your fine-tuned model
model_id = "ft:babbage-002:personal:kpi-1:9JbgNwVJ"

def query_model(prompt):
    try:
        response = openai.Completion.create(
            model=model_id,       # fine-tuned model
            prompt=prompt,        # input prompt
            max_tokens=100,        # output tokens
            temperature=0.5       # creativity level
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example prompt
prompt = '''Provide me Sentiment, Positive Aspects, Negative Aspects . Sentiment should only be positive/negative/neutral. List more than 1 aspect, but each aspect should be 3 words max. If there are no Positive/Negative Aspects provide the string x. The format is as so : 

Sentiment: {sentiment}

Aspect(Pos): {aspect_pos}

Aspect(Neg): {aspect_neg}   Do it for the following: These sardines are the best national brand - perfect condition, delicate flavor, and packed in pure olive oil. They make a great midnight snack on sourdough bread with mayo, sweet pickle relish, and Tabasco. Plus, they have a long shelf life, making them ideal for emergency food supplies.'''

# Get the response from the model
response_text = query_model(prompt)
print("Model Response:", response_text)


Model Response: Sentiment: Positive
Aspect(Pos): best national brand, perfect condition, delicate flavor, fresh, and ideal for emergency food supplies
Aspect(Neg): None
Aspect: fresh, and ideal for emergency food supplies
Aspect: best national brand, perfect condition, delicate flavor, fresh, and ideal for emergency food supplies
Aspect: emergency food supplies, fresh, and ideal for emergency food supplies
Aspect: best national brand, perfect condition, delicate flavor, fresh, and ideal for emergency food supplies


In [27]:
import re
model_results = """Sentiment: Positive
Aspect(Pos): best national brand, perfect condition, delicate flavor, and packed in oil
Aspect(Neg): None
Aspect: snack, emergency food supply, shelf life
Aspect(S): best national brand, perfect condition, delicate flavor, and oil
Aspect(N): None
Aspect(N): None

Aspect(Out): None

Aspect(Neg): None

Aspect(Out): None

Aspect(Out): None

Aspect(Out): None

Aspect(Out):"""

import pandas as pd
import re

def extract_sentiment_aspect(text):
    sentiment = re.search(r"Sentiment: (\w+)", text).group(1)
    aspect_pos = re.search(r"Aspect\(Pos\): (.+)", text).group(1)
    aspect_neg = re.search(r"Aspect\(Neg\): (.+)", text).group(1)
    return sentiment, aspect_pos, aspect_neg

# Sample model_results for demonstration, replace this with your actual data

# Extract sentiment and aspects
sentiment, aspect_pos, aspect_neg = extract_sentiment_aspect(model_results)

# Print the extracted values
print("Sentiment:", sentiment)
print("Aspect(Pos):", aspect_pos)
print("Aspect(Neg):", aspect_neg)

# Define columns for the DataFrame
columns = ['Sentiment', 'Aspect(Pos)', 'Aspect(Neg)']

# Create an empty DataFrame
dftest = pd.DataFrame(columns=columns)

# Create a new row with extracted data
new_row = pd.DataFrame([{'Sentiment': sentiment, 'Aspect(Pos)': aspect_pos, 'Aspect(Neg)': aspect_neg}])

# Concatenate the new row to the existing DataFrame
dftest = pd.concat([dftest, new_row], ignore_index=True)

# Print the DataFrame
dftest


Sentiment: Positive
Aspect(Pos): best national brand, perfect condition, delicate flavor, and packed in oil
Aspect(Neg): None


,Sentiment,Aspect(Pos),Aspect(Neg)
0,Positive,"best national brand, perfect condition, delica...",None


### Inferencing using Babbage

In [19]:
score_df = pd.DataFrame()
score_df["summary"] = test_df["summary"]
score_df["sentiment"] = test_df['sentiment'] 
score_df["aspect_pos"] = test_df['aspect_pos']
score_df["aspect_neg"] = test_df['aspect_neg'] 

#save the results for comparison
score_df.head()

,summary,sentiment,aspect_pos,aspect_neg
86,The item was surprisingly thin! Despite orderi...,negative,refund,"skinny, size, disappointed, cost, shipping, ve..."
432,"As a coffee lover, I had high expectations for...",negative,x,"strange, carbonated, aftertasrte, taste"
799,"When I received the Lickety Stik, my dogs were...",positive,"healthy ingredients, value the price","sticky thumb, biscuits, one bottle"
417,"As someone who enjoys bold and strong coffee, ...",positive,"taste, bold, strong",x
678,I wasn't able to find them reliably anywhere a...,positive,"excitement, various flavor, weight loss, break...",x


In [30]:
openai.api_key = ''
model_id = "ft:babbage-002:personal:kpi-1:9JbgNwVJ"

def query_model(prompt):
    try:
        response = openai.Completion.create(
            model=model_id,       # fine-tuned model
            prompt=prompt,        # input prompt
            max_tokens=100,        # output tokens
            temperature=0.5       # creativity level
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def prepare_and_query_model(summary):
    prompt = f'''Provide me Sentiment, Positive Aspects, Negative Aspects . Sentiment should only be positive/negative/neutral. List more than 1 aspect, but each aspect should be 3 words max. If there are no Positive/Negative Aspects provide the string x. The format is as so : 
    
Sentiment: {{sentiment}}

Aspect(Pos): {{aspect_pos}}

Aspect(Neg): {{aspect_neg}}   Do it for the following: {summary}'''
    return query_model(prompt)

test_df['Model Response'] = test_df['summary'].apply(prepare_and_query_model)

test_df.to_excel('test_data_with_model_responses.xlsx')



In [29]:
test_df

,summary,sentiment,aspect_pos,aspect_neg,completion,prompt
86,The item was surprisingly thin! Despite orderi...,negative,refund,"skinny, size, disappointed, cost, shipping, ve...",Sentiment: Negative\nAspect(Pos): refund\nAspe...,"Provide me Sentiment, Positive Aspects, Negati..."
432,"As a coffee lover, I had high expectations for...",negative,x,"strange, carbonated, aftertasrte, taste",Sentiment: Negative\nAspect(Pos): x\nAspect(Ne...,"Provide me Sentiment, Positive Aspects, Negati..."
799,"When I received the Lickety Stik, my dogs were...",positive,"healthy ingredients, value the price","sticky thumb, biscuits, one bottle",Sentiment: Positive\nAspect(Pos): healthy ingr...,"Provide me Sentiment, Positive Aspects, Negati..."
417,"As someone who enjoys bold and strong coffee, ...",positive,"taste, bold, strong",x,"Sentiment: Positive\nAspect(Pos): taste, bold,...","Provide me Sentiment, Positive Aspects, Negati..."
678,I wasn't able to find them reliably anywhere a...,positive,"excitement, various flavor, weight loss, break...",x,"Sentiment: Positive\nAspect(Pos): excitement, ...","Provide me Sentiment, Positive Aspects, Negati..."
...,...,...,...,...,...,...
209,"I personally enjoyed this chocolate bar, findi...",positive,"sweet, smooth",not enough cocoa,"Sentiment: Positive\nAspect(Pos): sweet, smoot...","Provide me Sentiment, Positive Aspects, Negati..."
446,I actually enjoyed this product. While it fall...,positive,"taste fruity, organic",x,Sentiment: Positive\nAspect(Pos): taste fruity...,"Provide me Sentiment, Positive Aspects, Negati..."
763,I have been a fan of coconut water for over 2 ...,negative,"taste, quality, satisfaction","mold, taste, sickness, incident","Sentiment: Negative\nAspect(Pos): taste, quali...","Provide me Sentiment, Positive Aspects, Negati..."
523,"I don't own a microwave, so I improvised the c...",positive,"great discount price, free from wheat, free fr...",x,Sentiment: Positive\nAspect(Pos): great discou...,"Provide me Sentiment, Positive Aspects, Negati..."


In [33]:
df = pd.read_excel('test_data_with_model_responses.xlsx')

df

,Unnamed: 0,summary,sentiment,aspect_pos,aspect_neg,completion,prompt,Model Response
0,86,The item was surprisingly thin! Despite orderi...,negative,refund,"skinny, size, disappointed, cost, shipping, ve...",Sentiment: Negative\nAspect(Pos): refund\nAspe...,"Provide me Sentiment, Positive Aspects, Negati...",Sentiment: Negative\nAspect(Pos): x\nAspect(Ne...
1,432,"As a coffee lover, I had high expectations for...",negative,x,"strange, carbonated, aftertasrte, taste",Sentiment: Negative\nAspect(Pos): x\nAspect(Ne...,"Provide me Sentiment, Positive Aspects, Negati...",Sentiment: Negative\nAspect(Pos): coffee drink...
2,799,"When I received the Lickety Stik, my dogs were...",positive,"healthy ingredients, value the price","sticky thumb, biscuits, one bottle",Sentiment: Positive\nAspect(Pos): healthy ingr...,"Provide me Sentiment, Positive Aspects, Negati...",Sentiment: Positive\nAspect(Pos): training too...
3,417,"As someone who enjoys bold and strong coffee, ...",positive,"taste, bold, strong",x,"Sentiment: Positive\nAspect(Pos): taste, bold,...","Provide me Sentiment, Positive Aspects, Negati...",Sentiment: Negative\nAspect(Pos): x\nAspect(Ne...
4,678,I wasn't able to find them reliably anywhere a...,positive,"excitement, various flavor, weight loss, break...",x,"Sentiment: Positive\nAspect(Pos): excitement, ...","Provide me Sentiment, Positive Aspects, Negati...","Sentiment: Positive\nAspect(Pos): affordable, ..."
...,...,...,...,...,...,...,...,...
159,209,"I personally enjoyed this chocolate bar, findi...",positive,"sweet, smooth",not enough cocoa,"Sentiment: Positive\nAspect(Pos): sweet, smoot...","Provide me Sentiment, Positive Aspects, Negati...","Sentiment: Negative\nAspect(Pos): enjoyable, s..."
160,446,I actually enjoyed this product. While it fall...,positive,"taste fruity, organic",x,Sentiment: Positive\nAspect(Pos): taste fruity...,"Provide me Sentiment, Positive Aspects, Negati...","Sentiment: Positive\nAspect(Pos): healthy, tas..."
161,763,I have been a fan of coconut water for over 2 ...,negative,"taste, quality, satisfaction","mold, taste, sickness, incident","Sentiment: Negative\nAspect(Pos): taste, quali...","Provide me Sentiment, Positive Aspects, Negati...",Sentiment: Negative\nAspect(Pos): best-tasting...
162,523,"I don't own a microwave, so I improvised the c...",positive,"great discount price, free from wheat, free fr...",x,Sentiment: Positive\nAspect(Pos): great discou...,"Provide me Sentiment, Positive Aspects, Negati...",Sentiment: Positive\nAspect(Pos): discounted p...


Post Processing

In [35]:
import pandas as pd
import re

# Define a function to extract sentiment, positive aspects, and negative aspects
def extract_sentiment_aspect(text):
    sentiment_match = re.search(r"Sentiment: (\w+)", text)
    sentiment = sentiment_match.group(1) if sentiment_match else None
    aspect_pos_match = re.search(r"Aspect\(Pos\): (.+)", text)
    aspect_pos = aspect_pos_match.group(1) if aspect_pos_match else None
    aspect_neg_match = re.search(r"Aspect\(Neg\): (.+)", text)
    aspect_neg = aspect_neg_match.group(1) if aspect_neg_match else None
    return sentiment, aspect_pos, aspect_neg

# Read data from Excel file into DataFrame
df = pd.read_excel('test_data_with_model_responses.xlsx')

# Apply the extraction function to each row of the DataFrame and assign the results
df[['Sentiment', 'Aspect(Pos)', 'Aspect(Neg)']] = df['Model Response'].apply(
    lambda x: pd.Series(extract_sentiment_aspect(x)) if pd.notnull(x) else pd.Series([None, None, None])
)

# Print the DataFrame
print(df)


     Unnamed: 0                                            summary sentiment   
0            86  The item was surprisingly thin! Despite orderi...  negative  \
1           432  As a coffee lover, I had high expectations for...  negative   
2           799  When I received the Lickety Stik, my dogs were...  positive   
3           417  As someone who enjoys bold and strong coffee, ...  positive   
4           678  I wasn't able to find them reliably anywhere a...  positive   
..          ...                                                ...       ...   
159         209  I personally enjoyed this chocolate bar, findi...  positive   
160         446  I actually enjoyed this product. While it fall...  positive   
161         763  I have been a fan of coconut water for over 2 ...  negative   
162         523  I don't own a microwave, so I improvised the c...  positive   
163         717  For the past few years my Beagle, Boing, has e...  positive   

                                       

In [42]:
# We'll calculate BERT scores for sentiment, positive aspects, and negative aspects
import pandas as pd
from bert_score import score

# Function to compute BERT scores for comparing text pairs
def compute_bert_scores(cands, refs):
    # Ensure inputs are lists of strings
    cands = [str(cand) for cand in cands]
    refs = [str(ref) for ref in refs]
    
    # Compute BERT scores
    P, R, F1 = score(cands, refs, lang="en", verbose=True)
    return P.mean().item(), R.mean().item(), F1.mean().item()

# Fill NaN values in specified columns
df['Sentiment'].fillna('x', inplace=True)
df['Aspect(Pos)'].fillna('x', inplace=True)
df['Aspect(Neg)'].fillna('x', inplace=True)

# Again, verify that there are no NaN values left
print(df[['Sentiment', 'Aspect(Pos)', 'Aspect(Neg)']].isnull().sum())


sentiment_scores = compute_bert_scores(df['Sentiment'], df['sentiment'])
aspect_pos_scores = compute_bert_scores(df['Aspect(Pos)'], df['aspect_pos'])
aspect_neg_scores = compute_bert_scores(df['Aspect(Neg)'], df['aspect_neg'])


print("BERT Scores for Sentiment:", sentiment_scores)
print("BERT Scores for Positive Aspects:", aspect_pos_scores)
print("BERT Scores for Negative Aspects:", aspect_neg_scores)

df['Combined'] = df['Sentiment'] + ' ' + df['Aspect(Pos)'] + ' ' + df['Aspect(Neg)']
df['combined'] = df['sentiment'] +' ' + df["aspect_pos"] + ' ' + df["aspect_neg"]
# Fill NaN values in the new column
df['Combined'].fillna('', inplace=True)

# Verify that there are no NaN values left
print(df['Combined'].isnull().sum())

# Compute BERT scores for the combined column
combined_scores = compute_bert_scores(df['Combined'], df['combined'])

print("BERT Scores for Combined Text:", combined_scores)

Sentiment      0
Aspect(Pos)    0
Aspect(Neg)    0
dtype: int64


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  1.09it/s]


computing greedy matching.


...

100%|██████████| 3/3 [00:37<00:00, 12.58s/it]


computing greedy matching.


100%|██████████| 3/3 [00:00<00:00, 50.78it/s]


done in 37.81 seconds, 4.34 sentences/sec
BERT Scores for Sentiment: (0.9755962491035461, 0.9746879935264587, 0.9751262068748474)
BERT Scores for Positive Aspects: (0.8616342544555664, 0.8667250871658325, 0.8638384938240051)
BERT Scores for Negative Aspects: (0.8031543493270874, 0.8395193219184875, 0.8199167847633362)
0


KeyError: 'combined'

In [43]:
df['Combined'] = df['Sentiment'] + ' ' + df['Aspect(Pos)'] + ' ' + df['Aspect(Neg)']
df['combined'] = df['sentiment'] +' ' + df["aspect_pos"] + ' ' + df["aspect_neg"]
# Fill NaN values in the new column
df['Combined'].fillna('', inplace=True)

# Verify that there are no NaN values left
print(df['Combined'].isnull().sum())

# Compute BERT scores for the combined column
combined_scores = compute_bert_scores(df['Combined'], df['combined'])

print("BERT Scores for Combined Text:", combined_scores)

0


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 6/6 [00:51<00:00,  8.58s/it]


computing greedy matching.


100%|██████████| 3/3 [00:00<00:00, 40.85it/s]

done in 51.58 seconds, 3.18 sentences/sec
BERT Scores for Combined Text: (0.8166630268096924, 0.8530115485191345, 0.8338488340377808)


In [45]:
import plotly.graph_objects as go

# BERT F1 Scores for different categories
categories = ['Sentiment', 'Positive Aspects', 'Negative Aspects', 'Combined Text']
f1_scores = [0.9751262068748474, 0.8638384938240051, 0.8199167847633362, 0.8338488340377808]

# Create a bar chart
fig = go.Figure([go.Bar(x=categories, y=f1_scores, marker_color='aqua')])

# Update layout
fig.update_layout(title='BERT F1 Scores for Various Text Categories',
                  xaxis_title='Text Categories',
                  yaxis_title='F1 Score',
                  yaxis=dict(range=[0.7, 1]))  # Adjust y-axis to better showcase differences

# Show the figure
fig.show()
